In [ ]:
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -p /usr/local -f

In [ ]:
! conda install -c milagraph -c conda-forge graphvite python=3.7 cudatoolkit=10.1 -y 

In [ ]:
!conda install -y wurlitzer ipykernel

In [ ]:
import site
site.addsitedir("/usr/local/lib/python3.7/site-packages")
%reload_ext wurlitzer

import torch
from graphvite.application.network import LinkPredictor

In [ ]:
def get_score(model, H, T):
    '''
    Get the un-normalized score (similarity) for `H` and `T` 
    within the given `model`
    '''
    model = LinkPredictor('LINE', model.solver.vertex_embeddings, model.solver.context_embeddings)
    model = model
    tensorH = torch.as_tensor(H)
    tensorT = torch.as_tensor(T)
    return model(tensorH, tensorT)

In [ ]:
cd drive/MyDrive/COMP90051/

/content/drive/MyDrive/COMP90051


In [ ]:
! graphvite run my_config.yml

loading graph from data/full.txt
0.0021379%
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
Graph<uint32>
------------------ Graph -------------------
#vertex: 3767, #edge: 29620
as undirected: yes, normalization: no
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
[time] GraphApplication.load: 0.00977302 s
[time] GraphApplication.build: 0.312915 s
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
GraphSolver<128, float32, uint32>
----------------- Resource -----------------
#worker: 1, #sampler: 1, #partition: 1
tied weights: no, episode size: 200
gpu memory limit: 14.8 GiB
gpu memory cost: 45 MiB
----------------- Sampling -----------------
augmentation step: 5, shuffle base: 5
random walk length: 60
random walk batch size: 100
#negative: 1, negative sample exponent: 0.75
----------------- Training -----------------
model: LINE
optimizer: SGD
learning rate: 0.025, lr schedule: linear
weight decay: 0.005
#epoch: 2000, batch size: 100000
resume: no
positive reuse: 1, negative weight: 5
>>>>>>>>>>>

In [ ]:
import pickle

dim = 32
length = 40
method = 'line'
directed = False
emb = f'embedding/dim{dim}-len{length}/{"directed_" if directed else ""}{method}_full.pkl'

model = pickle.load(open(emb, 'rb'))

In [ ]:
H, T = [], []
mapping = model.graph.name2id
with open('data/test-public.csv', 'r') as f:
    next(f)
    for line in f:
        _, h, t = line.split(',')
        h = str(int(h))
        t = str(int(t))
        if h in mapping:
            H.append(mapping[h])
        else:
            H.append(999)
        if t in mapping:
            T.append(mapping[t])
        else:
            T.append(999)


In [ ]:
score = get_score(model, H, T)
# normalize the scores to probabilities
smin = score.min(0, keepdim=True)[0]
smax = score.max(0, keepdim=True)[0]
score -= smin
score /= (smax - smin)

In [ ]:
print(score)

tensor([0.5411, 0.1135, 0.2669,  ..., 0.3603, 0.7027, 0.2649])


In [ ]:
# save the output
out = f'output/dim{dim}-len{length}_{"" if directed else "un"}directed_{method}.csv'
out = 'random_walk1.csv'
with open(out, 'w') as o:
    o.write('Id,Predicted\n')
    for i, s in enumerate(score.tolist()):
        o.write(f'{i+1},{s}\n')
print('Write output to', out)

Write output to random_walk1.csv
